**PLEASE NOTE:** This model was trained on only 1096 New York City images, hence performs best there - potentially could be retrained over a wider area.

See training images here: https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/maps.tar.gz

Model Code + Paper: https://github.com/affinelayer/pix2pix-tensorflow

Examples of a good prediction (from New York City):

input | output | target
--- | --- | ---
![](pix2pix_examples/20180108_161824_0-inputs.png) | ![](pix2pix_examples/20180108_161824_0-outputs.png) | ![](pix2pix_examples/20180108_161824_0-targets.png)

In [2]:
# WARNING: Run cell below first
display(button_clear_points)
display(button_run_prediction)
display(button_go_to_NYC)
display(zoom_level)

Button(description=u'Clear All Points', style=ButtonStyle())

Button(description=u'Run Prediction', style=ButtonStyle())

Button(description=u'Go to New York City', style=ButtonStyle())

IntSlider(value=16, continuous_update=False, description=u'Zoom level:', max=18, min=14)

Using Zoom = 16
Using lat, lon = 40.7508637076, -73.9694835614
Running predictions...


name | input | output | target
--- | --- | --- | ---
20180108_162520_0 | ![Picture](results/images/20180108_162520_0-inputs.png) | ![Picture](results/images/20180108_162520_0-outputs.png) | ![Picture](results/images/20180108_162520_0-targets.png) | 


In [1]:
# CODE:
# https://github.com/affinelayer/pix2pix-tensorflow
# https://github.com/OpenGeoscience/geonotebook/blob/eaeadd4070563ce4788fc167951945c1ab5bb079/notebooks/02_Raster_Data.ipynb

# FOR SATALITE BASEMAP:
# docker exec -it bfc1418aa28b bash
# apt-get install nano
# nano /usr/local/etc/geonotebook.ini
# add (many basemaps available at http://leaflet-extras.github.io/leaflet-providers/preview/):
#  url = https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}
#  attribution =Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community
# Restart Kernel
#
# NOTES:
#!git clone https://github.com/affinelayer/pix2pix-tensorflow.git
#!mv pix2pix-tensorflow/* .
#cp -r pretrained_models/ ~/git/geonotebook/notebooks/
#
#!apt-get update
#!apt-get install -y python3-pip

import os
import Image
import urllib
import shutil
from time import gmtime, strftime
from IPython.display import display
from io import BytesIO

from bs4 import BeautifulSoup
from geonotebook.wrappers import RasterData, RasterDataCollection
from motionless import CenterMap
from matplotlib import pylab as plt
%matplotlib inline


google_maps_api_key = "YOUR API KEY"


def clean_dir(dir):
    shutil.rmtree(dir, ignore_errors=True, onerror=None)
    
    try: 
        os.makedirs(dir)
    except OSError:
        if not os.path.isdir(dir):
            raise

            
def download_google_maps_from_geonb_points():
    if len(M.layers.annotation.points) == 0:
        raise ValueError, "Please add at least one point on the map."
    else:
        clean_dir("roadmap")
        clean_dir("satellite")
        
        print "Using Zoom = {}".format(zoom_level.value)
        for idx, point in enumerate(M.layers.annotation.points):
            lon, lat = point.x, point.y
            now = strftime("%Y%m%d_%H%M%S", gmtime()) + "_{}".format(idx)
            
            print "Using lat, lon = {0}, {1}".format(lat, lon)
            download_one_google_map(now, lat, lon, maptype="roadmap", zoom_level=zoom_level.value)
            download_one_google_map(now, lat, lon, maptype="satellite", zoom_level=zoom_level.value)
    
    

def download_one_google_map(now, lat, lon, maptype, zoom_level=16):
    cmap = CenterMap(lat=round(lat, 5),
                     lon=round(lon, 5),
                     maptype=maptype,
                     zoom = 16,
                     size_x=256,
                     size_y=256)
    url = cmap.generate_url()
    url = url + "&key=" + google_maps_api_key
    
    #print url
    if maptype == "satellite":
        out_path = "satellite/{}.png".format(now)
    elif maptype == "roadmap":
        out_path = "roadmap/{}.png".format(now)
    
    buffer = BytesIO(urllib.urlopen(url).read())
    image = Image.open(buffer)
    image.save(out_path)
    
    #print "Downloaded {}".format(out_path)
    

# See: https://github.com/WangYihang/HTML2Markdown/blob/master/HTML2Python.py

def handle_ths(ths):
    row_string = ""
    
    row_string += ' | '.join(str(e).replace("<th>", "").replace("</th>","") for e in ths)
    row_string += "\n"
    
    row_string += ' | '.join(["---" for _ in range(len(ths))])
    row_string += "\n"
    return row_string


def handle_img(img):
    describe = ""
    if img.has_attr('alt'):
        describe = img['alt']
    else:
        describe = "Picture"
    return "![" + describe + "](" + img['src'].replace("images", "results/images") + ")"


def handle_tds(tds):
    row_string = ""
        
    for td in tds:
        if "img" in str(td):
            row_string += handle_img(td.find_all("img")[0]) + " | "
        else:
            row_string += str(td).replace("<td>","").replace("</td>","") + " | "

    row_string += "\n"
    return row_string


def html2md(html):
    soup = BeautifulSoup(html, "html.parser")
    table_rows = soup.find_all("tr")

    md = ""
    
    header_row = True
    for table_row in table_rows:
        if header_row:
            header_row = False
            md += handle_ths(table_row.find_all("th"))
        else:
            md += handle_tds(table_row.find_all("td"))
            
    return md


def display_results():
    with open("results/index.html", "r") as results_index:
        result_html = results_index.read()

    result_md = html2md(result_html)

    from IPython.display import display, Markdown, Latex
    display(Markdown(result_md))
    
    
def clear_points():
    # Clear any annotations
    M.layers.annotation.clear_annotations()
    
    
def run_prediction():
    # Download google map+sat
    download_google_maps_from_geonb_points()

    # Create combine image
    clean_dir("combined")
    ! python3 tools/process.py --input_dir satellite --b_dir roadmap --operation combine --output_dir combined 2>&1 >/dev/null | grep -v "feature_guard.cc:137"

    # Run Pix2pix
    print "Running predictions..."
    clean_dir("results")
    ! python3 pix2pix.py --mode test --output_dir results --input_dir combined --checkpoint pretrained_models 2>&1 >/dev/null  | grep -v "feature_guard.cc:137"

    display_results()
    
    
import ipywidgets as widgets
from IPython.display import clear_output

button_clear_points = widgets.Button(description="Clear All Points")
button_run_prediction = widgets.Button(description="Run Prediction")
button_go_to_NYC = widgets.Button(description="Go to New York City")
zoom_level = widgets.IntSlider(
    value=16,
    min=14,
    max=18,
    step=1,
    description='Zoom level:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


def reset_buttons():
    clear_output()
    display(button_clear_points)
    display(button_run_prediction)
    display(button_go_to_NYC)
    display(zoom_level)

def on_button_clear_points_clicked(b):
    reset_buttons()
    clear_points()
    
def on_button_run_prediction_clicked(b):
    reset_buttons()
    run_prediction()

def on_button_go_to_NYC_clicked(b):
    # Zoom to New YorK City (Manhattan) - NYC is where the training data is
    M.set_center(-73.9712, 40.7500, 18)

on_button_go_to_NYC_clicked("b")
button_clear_points.on_click(on_button_clear_points_clicked)
button_run_prediction.on_click(on_button_run_prediction_clicked)
button_go_to_NYC.on_click(on_button_go_to_NYC_clicked)